In [1]:
pip install opencv-python numpy pydub moviepy pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 29.4 MB/s eta 0:00:00


In [2]:
import cv2
import numpy as np
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import base64
import os
from google.colab.patches import cv2_imshow  # Import this for displaying images in Colab

# Helper function to generate a 16-byte AES key from the user-provided key
def generate_aes_key(key):
    return pad(key.encode('utf-8'), 16)[:16]  # Ensure the key is 16 bytes long

# Function to encrypt the message using AES
def encrypt_message(message, key):
    aes_key = generate_aes_key(key)
    cipher = AES.new(aes_key, AES.MODE_ECB)
    encrypted_message = cipher.encrypt(pad(message.encode('utf-8'), AES.block_size))
    return base64.b64encode(encrypted_message).decode('utf-8')

# Function to decrypt the message using AES
def decrypt_message(encrypted_message, key):
    aes_key = generate_aes_key(key)
    cipher = AES.new(aes_key, AES.MODE_ECB)
    encrypted_message_bytes = base64.b64decode(encrypted_message.encode('utf-8'))
    decrypted_message = unpad(cipher.decrypt(encrypted_message_bytes), AES.block_size)
    return decrypted_message.decode('utf-8')

# Function to embed the encrypted message in the image using LSB
def encode_image(image_path, message, key, output_path):
    # Encrypt the message using AES
    encrypted_message = encrypt_message(message, key)
    print(f"Encrypted message: {encrypted_message}")

    # Read the image
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Image not found")

    # Convert the encrypted message to binary
    encrypted_message += '#####'  # Delimiter to mark the end of the message
    binary_message = ''.join([format(ord(char), '08b') for char in encrypted_message])

    # Encode the binary message into the image
    data_index = 0
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            pixel = list(image[i, j])
            for n in range(3):  # Iterate through R, G, B channels
                if data_index < len(binary_message):
                    # Modify the LSB of each color channel
                    pixel[n] = int(format(pixel[n], '08b')[:-1] + binary_message[data_index], 2)
                    data_index += 1
            image[i, j] = tuple(pixel)
            if data_index >= len(binary_message):
                break

    # Save the output image
    cv2.imwrite(output_path, image)
    #print(f"Message encoded into {output_path}")
    return "Image Encoded and saved successfully "
    cv2_imshow(image)  # Display the encoded image in Colab

# Function to extract and decrypt the message from the image
def decode_image(image_path, key):
    # Read the image
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Image not found")

    # Extract the binary message from the image
    binary_message = ""
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            pixel = image[i, j]
            for n in range(3):
                binary_message += format(pixel[n], '08b')[-1]

    # Convert the binary message to text
    message = ''.join([chr(int(binary_message[i:i + 8], 2)) for i in range(0, len(binary_message), 8)])
    encrypted_message = message.split('#####')[0]  # Extract the message up to the delimiter

    # Decrypt the message using AES
    try:
        decrypted_message = decrypt_message(encrypted_message, key)
        print(f"Decrypted message: {decrypted_message}")
        return decrypted_message
    except (ValueError, KeyError):
        print("Decryption failed. Invalid key or corrupted message.")
        return None

In [ ]:
#input_image_path = 'Test Image.png'  # Path to the input image
#output_image_path = 'encoded_image.png'  # Path to save the encoded image
#secret_message = "This is a top secret message!"
#encryption_key = "my_secret_key"  # User-provided key for AES encryption
#encode_image(input_image_path, secret_message, encryption_key, output_image_path)

In [ ]:
#output_image_path = 'encoded_image.png'
#decoding_key = input('Enter decryption key:')
#decoded_message = decode_image(output_image_path, decoding_key)

In [ ]:
pip install pydub

In [3]:
import wave
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import base64
from pydub import AudioSegment


# Helper function to generate a 16-byte AES key from the user-provided key
def generate_aes_key(key):
    return pad(key.encode('utf-8'), 16)[:16]  # Ensure the key is 16 bytes long

# Function to encrypt the message using AES
def encrypt_message(message, key):
    aes_key = generate_aes_key(key)
    cipher = AES.new(aes_key, AES.MODE_ECB)
    encrypted_message = cipher.encrypt(pad(message.encode('utf-8'), AES.block_size))
    return base64.b64encode(encrypted_message).decode('utf-8')

# Function to decrypt the message using AES
def decrypt_message(encrypted_message, key):
    aes_key = generate_aes_key(key)
    cipher = AES.new(aes_key, AES.MODE_ECB)
    encrypted_message_bytes = base64.b64decode(encrypted_message.encode('utf-8'))
    decrypted_message = unpad(cipher.decrypt(encrypted_message_bytes), AES.block_size)
    return decrypted_message.decode('utf-8')


def convert_mp3_to_wav(mp3_path):
    audio = AudioSegment.from_mp3(mp3_path)
    wav_path = mp3_path.replace(".mp3", ".wav")
    audio.export(wav_path, format="wav")
    return wav_path

# Function to embed the encrypted message in the audio using LSB
def encode_audio(audio_path, message, key, output_path):
    # Encrypt the message using AES

    if audio_path.endswith(".mp3"):
        audio_path = convert_mp3_to_wav(audio_path)
        print(f"Converted MP3 to WAV: {audio_path}")

    encrypted_message = encrypt_message(message, key)
    print(f"Encrypted message: {encrypted_message}")

    # Open the audio file
    audio = wave.open(audio_path, mode='rb')
    frame_bytes = bytearray(list(audio.readframes(audio.getnframes())))

    # Convert the encrypted message to binary
    encrypted_message += '#####'  # Delimiter to mark the end of the message
    binary_message = ''.join([format(ord(char), '08b') for char in encrypted_message])

    # Embed the binary message into the LSB of audio frames
    data_index = 0
    for i in range(len(frame_bytes)):
        if data_index < len(binary_message):
            frame_bytes[i] = (frame_bytes[i] & 254) | int(binary_message[data_index])
            data_index += 1

    # Save the modified audio
    modified_audio = wave.open(output_path, 'wb')
    modified_audio.setparams(audio.getparams())
    modified_audio.writeframes(bytes(frame_bytes))
    audio.close()
    modified_audio.close()
    #print(f"Message encoded into {output_path}")
    return "Audio Encoded and saved successfully "

# Function to extract and decrypt the message from the audio
def decode_audio(audio_path, kay):
    # Open the audio file
    audio = wave.open(audio_path, mode='rb')
    frame_bytes = bytearray(list(audio.readframes(audio.getnframes())))

    # Extract the binary message
    binary_message = ''.join([str(frame_bytes[i] & 1) for i in range(len(frame_bytes))])

    # Convert the binary message to text
    message = ''.join([chr(int(binary_message[i:i + 8], 2)) for i in range(0, len(binary_message), 8)])
    encrypted_message = message.split('#####')[0]  # Extract the message up to the delimiter
    audio.close()

    # Decrypt the message using AES
    try:
        decrypted_message = decrypt_message(encrypted_message, kay)
        print(f"Decrypted message: {decrypted_message}")
        return decrypted_message
    except (ValueError, KeyError):
        print("Decryption failed. Invalid key or corrupted message.")
        return None

In [ ]:
#input_audio_path = 'Sample_Wav.wav'  # Path to the input audio file
#output_audio_path = 'encoded_audio.wav'  # Path to save the encoded audio
#secret_message = input("Enter the secret message to encode: ")
#encryption_key = input("Enter the encryption key: ")  # User-provided key for AES encryption
#encode_audio(input_audio_path, secret_message, encryption_key, output_audio_path)

Enter the secret message to encode: Hassan
Enter the encryption key: 123123
Encrypted message: 6/k8shSWDazzDMRs3MK/zg==
Message encoded into encoded_audio.wav


In [ ]:
#print("\nDecoding the message from the encoded audio...")
#output_audio_path = 'Eco_audio.wav'
#key=input("Enter the decryption key:")
#decoded_message = decode_audio(output_audio_path,key)


Decoding the message from the encoded audio...
Enter the decryption key:123123
Decrypted message: YEssssssssss


In [4]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import base64

# Helper function to generate a 16-byte AES key from the user-provided key
def generate_aes_key(key):
    return pad(key.encode('utf-8'), 16)[:16]  # Ensure the key is 16 bytes long

# Function to encrypt the message using AES
def encrypt_message(message, key):
    aes_key = generate_aes_key(key)
    cipher = AES.new(aes_key, AES.MODE_ECB)
    encrypted_message = cipher.encrypt(pad(message.encode('utf-8'), AES.block_size))
    return base64.b64encode(encrypted_message).decode('utf-8')

# Function to decrypt the message using AES
def decrypt_message(encrypted_message, key):
    aes_key = generate_aes_key(key)
    cipher = AES.new(aes_key, AES.MODE_ECB)
    encrypted_message_bytes = base64.b64decode(encrypted_message.encode('utf-8'))
    decrypted_message = unpad(cipher.decrypt(encrypted_message_bytes), AES.block_size)
    return decrypted_message.decode('utf-8')

# Function to embed the encrypted message in the text using whitespace
def encode_text(text, message, key):
    # Remove spaces from the original text
    text = text.replace(" ", "")

    # Encrypt the message using AES
    encrypted_message = encrypt_message(message, key)
    print(f"Encrypted message: {encrypted_message}")

    # Convert the encrypted message to binary
    encrypted_message += '#####'  # Delimiter to mark the end of the message
    binary_message = ''.join([format(ord(char), '08b') for char in encrypted_message])

    # Embed the binary message into the text using spaces ('0') and tabs ('1')
    encoded_text = ""
    data_index = 0
    for char in text:
        encoded_text += char
        if data_index < len(binary_message):
            if binary_message[data_index] == '0':
                encoded_text += ' '  # Use space to represent '0'
            else:
                encoded_text += '\t'  # Use tab to represent '1'
            data_index += 1

    # Add any remaining binary message if the original text runs out
    while data_index < len(binary_message):
        if binary_message[data_index] == '0':
            encoded_text += ' '
        else:
            encoded_text += '\t'
        data_index += 1

    return encoded_text

# Function to extract and decrypt the message from the encoded text
def decode_text(encoded_text):
    # Extract the binary message from spaces ('0') and tabs ('1')
    binary_message = ""
    for char in encoded_text:
        if char == ' ':
            binary_message += '0'
        elif char == '\t':
            binary_message += '1'

    # Convert the binary message to text
    try:
        message = ''.join([chr(int(binary_message[i:i + 8], 2)) for i in range(0, len(binary_message), 8)])
        encrypted_message = message.split('#####')[0]  # Extract the message up to the delimiter
    except ValueError:
        print("Failed to extract the message. The encoded text may be corrupted.")
        return None

    # Ask the user for the decryption key
    key = input("Enter the decryption key: ")

    # Decrypt the message using AES
    try:
        decrypted_message = decrypt_message(encrypted_message, key)
        print(f"Decrypted message: {decrypted_message}")
        return decrypted_message
    except (ValueError, KeyError):
        print("Decryption failed. Invalid key or corrupted message.")
        return None

In [5]:
original_text = input("Enter the original text to encode the message in: ")
secret_message = input("Enter the secret message to encode: ")
encryption_key = input("Enter the encryption key: ")  # User-provided key for AES encryption

encoded_text = encode_text(original_text, secret_message, encryption_key)
print("\nEncoded Text:")
print(encoded_text)

Enter the original text to encode the message in: Testing
Enter the secret message to encode: haha
Enter the encryption key: 123123
Encrypted message: dcEg6HYeNx8N92M6jRP49Q==

Encoded Text:
T e	s	t i n	g   		   		 	   	 	 		  			  		 		  	  	    	 		  	 		  	 	 	  			  				     			    	  			   			  	  		  	  	  		 	  		 		  		 	 	  	 	  	  	 	      		 	    			  	 	 	   	  				 	  				 	  	   		  	   		  	   		  	   		  	   		


In [6]:
print("\nDecoding the message from the encoded text...")
decoded_message = decode_text(encoded_text)


Decoding the message from the encoded text...
Enter the decryption key: 123123
Decrypted message: haha


In [ ]:
import cv2
import numpy as np
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import base64

def generate_aes_key(key):
    if isinstance(key, str):
        key = key.encode('utf-8')
    return pad(key, AES.block_size)[:AES.block_size]

def encrypt_message(message, key):
    aes_key = generate_aes_key(key)
    cipher = AES.new(aes_key, AES.MODE_ECB)
    padded_message = pad(message.encode('utf-8'), AES.block_size)
    encrypted_message = cipher.encrypt(padded_message)
    return base64.b64encode(encrypted_message).decode('utf-8')

def decrypt_message(encrypted_message, key):
    try:
        aes_key = generate_aes_key(key)
        cipher = AES.new(aes_key, AES.MODE_ECB)
        decrypted_message = cipher.decrypt(base64.b64decode(encrypted_message))
        unpadded_message = unpad(decrypted_message, AES.block_size)
        return unpadded_message.decode('utf-8')
    except Exception:
        return "Cannot decrypt"

def extract_frames(video_path):
    video = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = video.read()
        if not ret:
            break
        frames.append(frame)
    video.release()
    return frames

def embed_message(frames, encrypted_message):
    message_bits = ''.join(format(ord(c), '08b') for c in encrypted_message)
    bit_index = 0
    for frame in frames:
        for j in range(frame.shape[0]):
            for k in range(frame.shape[1]):
                if bit_index >= len(message_bits):
                    return frames
                pixel = frame[j, k]
                for l in range(3):  # RGB channels
                    if bit_index >= len(message_bits):
                        break
                    # Modify the least significant bit of the pixel value
                    pixel[l] = (pixel[l] & 254) | int(message_bits[bit_index])
                    bit_index += 1
    return frames

def extract_message_from_video(video_path, message_length):
    video = cv2.VideoCapture(video_path)
    extracted_bits = ''
    while True:
        ret, frame = video.read()
        if not ret or len(extracted_bits) >= message_length * 8:
            break
        for j in range(frame.shape[0]):
            for k in range(frame.shape[1]):
                pixel = frame[j, k]
                for l in range(3):  # RGB channels
                    extracted_bits += str(pixel[l] & 1)
                    if len(extracted_bits) >= message_length * 8:
                        video.release()
                        return extracted_bits
    video.release()
    return extracted_bits

def save_video(frames, output_path, fps=30):
    height, width, _ = frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'FFV1')  # Lossless codec (FFV1)
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    for frame in frames:
        out.write(frame)
    out.release()

In [ ]:
!pip install gradio


In [ ]:
encrypted_message_global = None

In [ ]:
print(encrypted_message_global)

owjKfy4nyN8/vZZBWT9vHg==


In [ ]:
def encode_video(input_video, secret_message, key, output_video):

    global encrypted_message_global

    # Extract frames from the input video
    original_frames = extract_frames(input_video)

    # Encrypt the secret message
    encrypted_message = encrypt_message(secret_message, key)
    encrypted_message_global = encrypted_message
    print(f"Encrypted message: {encrypted_message}")

    # Embed the encrypted message into the frames
    stego_frames = embed_message(original_frames, encrypted_message)

    # Save the encoded frames back as a video
    save_video(stego_frames, output_video)
    print(f"Message encoded into {output_video}")
    return "Video encoding completed successfully!"

def decode_video(output_video, kay):
  print("Decoding the message from the encoded video...")
  global encrypted_message_global
  encrypted_message=encrypted_message_global

  extracted_bits = extract_message_from_video(output_video, len(encrypted_message))

  if extracted_bits:
     extracted_message = ''.join(chr(int(extracted_bits[i:i+8], 2)) for i in range(0, len(extracted_bits), 8))
     decrypted_message = decrypt_message(extracted_message, kay)
     return decrypted_message
     if decrypted_message:
        return decrypted_message
     else:
        print("Failed to decrypt the message")
  else:
    print("No message could be extracted from the video")


In [ ]:
import gradio as gr
from pathlib import Path

# File type options
FILE_TYPES = ["Image", "Audio", "Video"]

# Encoding logic
def hide_message(file_type, file_path, secret_message, encryption_key):
    if file_path is None:
        return "Please upload a file."

    # Generate output file name based on type
    input_path = Path(file_path.name)  # Get uploaded file's name
    if file_type == "Image":
        output_file = str(input_path.stem) + "_encoded.png"  # Force .png extension
        return encode_image(file_path, secret_message, encryption_key, output_file)
    elif file_type == "Audio":
        output_file = str(input_path.stem) + "_encoded.wav"  # Force .wav extension
        return encode_audio(file_path, secret_message, encryption_key, output_file)
    elif file_type == "Video":
        output_file = str(input_path.stem) + "_encoded.avi"  # Force .avi extension
        return encode_video(file_path, secret_message, encryption_key, output_file)
    else:
        return "Invalid file type."

# Decoding logic
def reveal_message(file_type, file_path, encryption_key):
    if file_path is None:
        return "Please upload a file."

    if file_type == "Image":
        return decode_image(file_path, encryption_key)
    elif file_type == "Audio":
        return decode_audio(file_path, encryption_key)
    elif file_type == "Video":
        return decode_video(file_path, encryption_key)
    else:
        return "Invalid file type."

# GUI setup
with gr.Blocks() as gui:
    gr.Markdown("# Steganography Tool for Image, Audio, and Video")

    with gr.Tabs():
        # Tab for Hiding Message
        with gr.Tab("Hide Message"):
            file_type_hide = gr.Radio(FILE_TYPES, label="Select File Type")
            file_path_hide = gr.File(label="Upload File")
            secret_message = gr.Textbox(label="Secret Message")
            encryption_key_hide = gr.Textbox(label="Encryption Key", type="password")
            encode_button = gr.Button("Encode Message")

            encode_result = gr.Textbox(label="Result")
            encode_button.click(hide_message,
                                inputs=[file_type_hide, file_path_hide, secret_message, encryption_key_hide],
                                outputs=encode_result)

        # Tab for Revealing Message
        with gr.Tab("Reveal Message"):
            file_type_reveal = gr.Radio(FILE_TYPES, label="Select File Type")
            file_path_reveal = gr.File(label="Upload File")
            encryption_key_reveal = gr.Textbox(label="Decryption Key", type="password")
            decode_button = gr.Button("Decode Message")

            decode_result = gr.Textbox(label="Decoded Message")
            decode_button.click(reveal_message,
                                inputs=[file_type_reveal, file_path_reveal, encryption_key_reveal],
                                outputs=decode_result)

# Launch the GUI
gui.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2e6dfc5caedd68dbf1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
